# Data

In [1]:
# libraries
# importing libraries
import pandas as pd
import numpy as np
import folium, json,requests, googlemaps, pprint, time
from geopy.geocoders import Nominatim
from sklearn import preprocessing
import itertools
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [2]:
# get categories functions
def get_venue_category_type(row):
    try:
        category_list = row["categories"]
    except:
        category_list = row["venue.categories"]
    if len(category_list) == 0:
        return None
    else:
        return category_list[0]["name"]

In [4]:
#getting location for the center point of the city
address = "Plaza 24 de Septiembre, Santa Cruz, Bolivia"
geolocator = Nominatim(user_agent="foursquare_agent")
geocode = geolocator.geocode(address)
client_id="G3WAOM5ZI0MEA2CQY53QKVYPBP24PLKX05VZNRQOFZ5EP01D"
client_secret="F31PEH03QREWU0D2JTOK0NKO3HGM3MRY3VQWMDBNQC5QY0AC"
v="20191206"
radius = 4000
search_query = "food"
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(client_id, client_secret, geocode.latitude, geocode.longitude, v, search_query, radius)
results = requests.get(url).json()
venues = pd.io.json.json_normalize(results["response"]["venues"])
# filtering useful columns
venues = venues.loc[slice(None), ["name", "categories"] + [col for col in venues.columns if col.startswith("location.")] + ["id"]]
venues["categories"] = venues.apply(get_venue_category_type, axis=1)
# Deleting useless columns
venues.drop(columns=["location.address", "location.crossStreet", "location.cc"], inplace=True)
# filling missing data and replacing to get more uniform data
venues["location.state"].fillna("Santa Cruz", inplace=True)
venues["location.state"].replace(to_replace=["SCZ", "Andres Ibañez"], value="Santa Cruz", inplace=True)
venues["location.city"].fillna("Santa Cruz de la Sierra", inplace=True)
venues["location.city"].replace(to_replace=["Santa Cruz", "Bolivia"], value="Santa Cruz de la Sierra", inplace=True)

/Users/sebastianplazaponteeguez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


In [5]:
venues

,name,categories,location.lat,location.lng,location.labeledLatLngs,location.distance,location.state,location.country,location.formattedAddress,location.city,id
0,Las Vegas Buffet Fast Food,Buffet,-17.773001,-63.194445,"[{'label': 'display', 'lat': -17.773001, 'lng'...",1738,Santa Cruz,Bolivia,[Bolivia],Santa Cruz de la Sierra,5d7bd58e2deabf00082c99db
1,Brazilian Fast Food,Fast Food Restaurant,-17.782999,-63.185392,"[{'label': 'display', 'lat': -17.782999, 'lng'...",348,Santa Cruz,Bolivia,"[Santa Cruz de la Sierra, Bolivia]",Santa Cruz de la Sierra,4d08be1ce554721e4f66c3f8
2,Chapaco's Fast Food Grill,BBQ Joint,-17.766326,-63.164481,"[{'label': 'display', 'lat': -17.7663256212280...",2657,Santa Cruz,Bolivia,"[Santa Cruz de la Sierra, Bolivia]",Santa Cruz de la Sierra,5349db8f498e54a3178975a3
3,"Canyella - Food, Drinks & Music",Mediterranean Restaurant,-17.782080,-63.186061,"[{'label': 'display', 'lat': -17.7820803075033...",439,Santa Cruz,Bolivia,"[Calle 21 de Mayo 258, Santa Cruz de la Sierra...",Santa Cruz de la Sierra,52b1dedd11d2564fa7a312a0
4,My Food,Fast Food Restaurant,-17.785425,-63.188457,"[{'label': 'display', 'lat': -17.7854251861572...",712,Santa Cruz,Bolivia,"[Av Cañoto, Santa Cruz de la Sierra, Bolivia]",Santa Cruz de la Sierra,55048e00498ee7519a90e643
5,Garajes Food Trucks,Food Truck,-17.755136,-63.172903,"[{'label': 'display', 'lat': -17.7551359812027...",3283,Santa Cruz,Bolivia,"[Santa Cruz de la Sierra, Bolivia]",Santa Cruz de la Sierra,5a419c6a6f0aa276862e5dfd
6,Mexican Food,Mexican Restaurant,-17.788620,-63.186354,"[{'label': 'display', 'lat': -17.7886199703056...",743,Santa Cruz,Bolivia,[Bolivia],Santa Cruz de la Sierra,4f95ecdbe4b0d1d5d4540221
7,La Estancia Fast Food,Argentinian Restaurant,-17.782375,-63.189168,"[{'label': 'display', 'lat': -17.7823749343877...",754,Santa Cruz,Bolivia,"[Santa Cruz de la Sierra, Bolivia]",Santa Cruz de la Sierra,4fc16687e4b0d516251295af
8,Mexican Food,Mexican Restaurant,-17.788543,-63.186412,"[{'label': 'display', 'lat': -17.7885434367881...",740,Santa Cruz,Bolivia,"[Av. Los cusis #2190 (Entre Beni y Alemana), S...",Santa Cruz de la Sierra,50fb48c3e4b0d0486d09b3f3
9,Toty's Fast Food,Fast Food Restaurant,-17.774745,-63.180204,"[{'label': 'display', 'lat': -17.7747451624698...",973,Santa Cruz,Bolivia,"[Avenida Uruguay #150, Santa Cruz de la Sierr...",Santa Cruz de la Sierra,524af9f611d21590ad9a2786
